<a href="https://colab.research.google.com/github/keonroohparvar/weatherRNNSurp/blob/main/Surp2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from pysolar.solar import *
from PIL import Image
from math import floor
import cv2
import numpy as np
import datetime
from openpyxl import Workbook, workbook
from openpyxl.utils import get_column_letter

workbook = Workbook()
sheet = workbook.active

# This file will process JPEG images using the opencv package

# Geographical Location of the Camera 

#latitude = 35.32
#longitude = -120.69

latitude = 42.206
longitude = -71.382

#Minutes passed since 12am. The first image is at 10am, so 9 hours  * 60 mintes per hour = 540 minutes have passed 
# in the day before the first image is taken. 
startTime = 540 


# Helper Function that returns a list of imagePath strings that are the images in the ./images folder
def loadImages(folderName):
    images = []

    numImages = len([name for name in os.listdir(f"./{folderName}")])

    # print(f"numImages is: {numImages}")

    for i in range(numImages):
        baseImg = f"img{i}.jpg"
        imgPath = os.path.join(folderName, baseImg)
        images.append(imgPath)

    return images


# Find Sun by comparing image to six differnet template images of the sun
def findSun(imagePath, templateImagesPath, imageCounter):
    # print("Finding Sun for Image {}...".format(imagePath))

    # Checks that dir is there
    if not os.path.isdir(templateImagesPath):
        print("Template images dir not correctly formatted.")

    # Open image
    image = cv2.imread(imagePath, 0)
    # print("Original Image Shape: {}".format(image.shape))

    #time of day in minutes (ie 11:00am = 600 min)
    curTime = startTime + 5 * imageCounter

    # Array to store image names
    imgNames = []
    for filename in os.listdir(templateImagesPath):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            imgNames.append(os.path.join(templateImagesPath, filename))

    # We will be using opencv's template matching to find the Sun. we will first compare the image to
    # template1.jpg, which is a clear view of the sun. We'll then go through and compare each template until we
    # find one that is pretty close.

    # Variable for formatting which template we are looking at
    # imgNum = i+1
    
    # Format image name to: {cwd}/src/template{imgNum}.jpg
    templateRootName = imgNames[0]
    templateRootName = templateRootName[:len(templateRootName) - 5]

    hour, min = convertMinToTime(curTime)

    naiveDate = datetime.datetime(2020, 5, 19, hour, minute, 0, 0)  
    timezone = pytz.timezone("America/Los_Angeles")
    # aware date now includes timezone info (accounts for daylight savings as well)
    date = timezone.localize(date)  
    azimuth = get_azimuth(latitude, longitude, date)
    altitude = get_altitude(latitude, longitude, date)
    print(azimuth)
    print(altitude)
    print("---") 
    highestConfidence = 0
    bestTemplate = 0
    bestmax_loc = 0
    templateCount = 1

    # Create copy of original image to edit
    imgBox = image.copy()

    #template 6 is always the best
    while templateCount <= 6:
        templateName = templateRootName + str(templateCount) + ".jpg"

        # Read template Image
        template = cv2.imread(templateName, 0)

        # Apply template matching to image with specified template
        result = cv2.matchTemplate(imgBox, template, cv2.TM_CCORR)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

       
        if max_val > highestConfidence:
            highestConfidence = max_val
            bestTemplate = template
            bestmax_loc = max_loc

        templateCount += 1

    # Assign the maximum equal value of template to variable topleft location
    topLeft = bestmax_loc

    # Find dimensions of template image for drawing the box
    width, height = bestTemplate.shape[::-1]

    # Calculate Dimensions of Box around object
    bottomRight = (topLeft[0] + width, topLeft[1] + height)

    # Find middle point of Sun
    middle = (int(topLeft[0]+(width/2)), int(topLeft[1] + (height / 2)))

    # Draw Box around object
    cv2.rectangle(imgBox, topLeft, bottomRight, 0, 2)

    # Draw Circle around Middle
    cv2.circle(imgBox, middle, 5, (0, 0, 0), -1)

    # Resize imgBox for better viewing
    newWidth = int(imgBox.shape[1]*50 / 100)
    newHeight = int(imgBox.shape[0]*50/100)
    smallerImg = cv2.resize(imgBox, (newWidth, newHeight))


    # print("Value is: {} and width:{} and height:{}".format(max_val, width, height))

    # print("Max location: ({},{})".format(max_loc[0], max_loc[1]))
    # print("Min Location: ({}, {})".format(min_loc[0], min_loc[1]))

    #cv2.imshow("Test", smallerImg)

    #cv2.waitKey()

    return middle

#Convert the min an image was taken and convert it to the hour and minute of the day
#helper function for using pysolar in findSun() 
def convertMinToTime(min):
    hour = (min / 60) + 1
    minute = (min % 60)
    return floor(hour), floor(minute)


# Function to create all 7 rings which we will be checking the haze index %'s of
def createRings(imagePath, middle):
    # Opening Image
    image = cv2.imread(imagePath, 0)
    imageColor = cv2.imread(imagePath, 1)

    # Parse height and width from image, and create empty masks for 7 rings
    height, width = image.shape
    # print("img: {} x {}".format(width, height))
    mask1 = np.zeros((height, width), np.uint8)
    mask2 = np.zeros((height, width), np.uint8)
    mask3 = np.zeros((height, width), np.uint8)
    mask4 = np.zeros((height, width), np.uint8)
    mask5 = np.zeros((height, width), np.uint8)
    mask6 = np.zeros((height, width), np.uint8)
    mask7 = np.zeros((height, width), np.uint8)

    # Value to determine Ring Seperation
    ringPixels = 200
    centerPixels = 70

    # Drawing Rings
    circle1Img = cv2.circle(mask1, middle, centerPixels, (255, 255, 255), thickness=-1)
    circle2Img = cv2.circle(mask2, middle, centerPixels + (1 * ringPixels), (255, 255, 255), thickness=-1)
    circle3Img = cv2.circle(mask3, middle, centerPixels + (2 * ringPixels), (255, 255, 255), thickness=-1)
    circle4Img = cv2.circle(mask4, middle, centerPixels + (3 * ringPixels), (255, 255, 255), thickness=-1)
    circle5Img = cv2.circle(mask5, middle, centerPixels + (4 * ringPixels), (255, 255, 255), thickness=-1)
    circle6Img = cv2.circle(mask6, middle, centerPixels + (5 * ringPixels), (255, 255, 255), thickness=-1)
    circle7Img = cv2.circle(mask7, middle, centerPixels + (6 * ringPixels), (255, 255, 255), thickness=-1)

    # Making inverses of each mask to 'Bitwise AND' with future Rings to seperate each ring. For example, Ring 5 should NOT contain any pixels from Ring 4, so
    # we will AND the Ring 5 mask with the inverse of the Ring 4 mask.
    mask1Inv = cv2.bitwise_not(mask1)
    ring1Mask = mask1

    mask2Inv = cv2.bitwise_not(mask2)
    ring2Mask = cv2.bitwise_and(circle2Img, mask1Inv)

    mask3Inv = cv2.bitwise_not(mask3)
    ring3Mask = cv2.bitwise_and(circle3Img, mask2Inv)

    mask4Inv = cv2.bitwise_not(mask4)
    ring4Mask = cv2.bitwise_and(circle4Img, mask3Inv)

    mask5Inv = cv2.bitwise_not(mask5)
    ring5Mask = cv2.bitwise_and(circle5Img, mask4Inv)

    mask6Inv = cv2.bitwise_not(mask6)
    ring6Mask = cv2.bitwise_and(circle6Img, mask5Inv)

    ring7Mask = cv2.bitwise_and(circle7Img, mask6Inv)

    # Create masked rings for all 7 rings by 'Bitwise AND'-ing each mask with the inverse of the previous mask.
    maskedData1 = cv2.bitwise_and(imageColor, imageColor, mask=circle1Img)
    maskedData2 = cv2.bitwise_and(imageColor, imageColor, mask=ring2Mask)
    maskedData3 = cv2.bitwise_and(imageColor, imageColor, mask=ring3Mask)
    maskedData4 = cv2.bitwise_and(imageColor, imageColor, mask=ring4Mask)
    maskedData5 = cv2.bitwise_and(imageColor, imageColor, mask=ring5Mask)
    maskedData6 = cv2.bitwise_and(imageColor, imageColor, mask=ring6Mask)
    maskedData7 = cv2.bitwise_and(imageColor, imageColor, mask=ring7Mask) 

    # Determine how many pixels are in each ring
    ringPixels = []
    ringPixels.append(np.count_nonzero(ring1Mask))
    ringPixels.append(np.count_nonzero(ring2Mask))
    ringPixels.append(np.count_nonzero(ring3Mask))
    ringPixels.append(np.count_nonzero(ring4Mask))
    ringPixels.append(np.count_nonzero(ring5Mask))
    ringPixels.append(np.count_nonzero(ring6Mask))
    ringPixels.append(np.count_nonzero(ring7Mask))

    # Create array of Ring Masks to use in the processImage() function
    ringMasks = []
    ringMasks.append(ring1Mask)
    ringMasks.append(ring2Mask)
    ringMasks.append(ring3Mask)
    ringMasks.append(ring4Mask)
    ringMasks.append(ring5Mask)
    ringMasks.append(ring6Mask)
    ringMasks.append(ring7Mask)
    ringMasks.append(ring7Mask)

    #print(ringPixels)

    #cv2.imshow("Ring 5 mask", ring5Mask)
    
    #cv2.waitKey()

    return [maskedData1, maskedData2, maskedData3, maskedData4, maskedData5, maskedData6, maskedData7], ringPixels, ringMasks


# Function to process images and calculate % values for each ring.
def processImage(imagePath, thresholdLow, thresholdHigh, counterImage, printImg=False):
    # Open Original Image
    # originalImage = cv2.imread(imagePath, cv2.IMREAD_UNCHANGED)
    # print(f"imagePath is {imagePath}")
    originalImage = cv2.imread(imagePath, 1)

    # Finding Sun
    middle = findSun(imagePath, './src/templateImages', counterImage)

    # Creating Rings and Counting the Pixels in each Ring
    rings, ringPixels, ringMasks = createRings(imagePath, middle)
    
    # Create Variable for Storing Percentages
    percentages = [0, 0, 0, 0, 0, 0, 0]

    finalImg = np.zeros(originalImage.shape, np.uint8)

    # Handles ring % calculation for each indiviual ring and saves it into percentages array
    for i in range(len(rings)):
        ring = rings[i]

        # cv2.imshow('ring{}'.format(i), ring)
        # cv2.waitKey()
        # cv2.destroyAllWindows()

        
        # Parse Ring into appropriate channels
        blueChannel = ring[:,:,0]
        greenChannel = ring[:,:,1]
        redChannel = ring[:,:,2]

        # Calculate numpy 2-d array of each pixel's haze index
        hazeChannel = (((redChannel + blueChannel) / 2) - greenChannel + 1) / (((redChannel + blueChannel) / 2) + greenChannel + 1)

        # Create mask using threshold values
        currentRingMask = cv2.inRange(hazeChannel, thresholdLow, thresholdHigh)

        # Need to 'Bitwise AND' each ring mask with the inverse of the mask above and below it to only leave the pixels in the specified ring
        currentRingMask = cv2.bitwise_and(currentRingMask, currentRingMask, mask=ringMasks[i])

        # See How many Pixels are Passed the Threshold in ringMask
        currentRingPixels = np.count_nonzero(currentRingMask)
        # print("Ring {} has {} nonzero pix out of {}, yielding {}%".format(i, currentRingPixels, ringPixels[i-1], (currentRingPixels / ringPixels[i-1])))

        # Calculate % covered and store in percentages
        percentages[i] = 100 - 100 * (currentRingPixels / ringPixels[i-1])
        percentages[i] = 0 if percentages[i] < 0 else percentages[i]


        frame = cv2.bitwise_and(originalImage, originalImage, mask=currentRingMask)

        # print("Final image shape: {}".format(finalImg.shape))
        # print("Frame Shape: {}".format(frame.shape))

        finalImg = cv2.add(finalImg, frame, 1)

    return percentages

        
    # Display the final Image
    if printImg:
        newWidth = int(finalImg.shape[1] * 50 / 100)
        newHeight = int(finalImg.shape[0] * 50 / 100)
        smallerFinalImg = cv2.resize(finalImg, (newWidth, newHeight))
        cv2.imshow("Final Image with threshold {} to {}".format(thresholdLow, thresholdHigh), smallerFinalImg)
        print("Percentages for Image {} are: {}".format(imagePath, percentages))
        cv2.waitKey()
        cv2.destroyAllWindows()


# Function to change names of Columns in worksheet 
def updateWorksheetNames():
    global sheet

    sheet["A1"] = "Ring 1 (Sun)"
    sheet["B1"] = "Ring 2"
    sheet["C1"] = "Ring 3"
    sheet["D1"] = "Ring 4"
    sheet["E1"] = "Ring 5"
    sheet["F1"] = "Ring 6"
    sheet["G1"] = "Ring 7"



# Function to save percentages in appropriate excell column
def savePercentages(percentages, rowNum):
    global sheet

    # This addition is to account for the 1st row being the Column Names 
    rowNum += 2

    
    sheet[f"A{rowNum}"] = percentages[0]
    sheet[f"B{rowNum}"] = percentages[1]
    sheet[f"C{rowNum}"] = percentages[2]
    sheet[f"D{rowNum}"] = percentages[3]
    sheet[f"E{rowNum}"] = percentages[4]
    sheet[f"F{rowNum}"] = percentages[5]
    sheet[f"G{rowNum}"] = percentages[6]


# Main Method
if __name__ == "__main__":
    
    print("Welcome to the process Image shell.")
    currOption = ""
    
    while currOption != "q":
        # Print options for user in Shell
        print("\nOptions:\n\tp folderPath --- Process all images in specified folderPath")
        print("\td --- Display potential folder paths for processing")
        print("\tq --- Quit the shell\n")
        
        # Parse input
        rawInput = input("\033[1;32;40mprocessshell\033[1;0;0m$ ")
        inputList = rawInput.split(" ")
        currOption = inputList[0]
        if len(inputList) > 0:
            params = inputList[1:]  
        else:
            params = ["1"]
        
        # Handle p option used for processing the data at CWD + /folderPath
        if currOption == "p":
            folderPath = params[0]
            
            if folderPath == "1":
                print("Using folerpath images/2021/3-4...")
                folderPath = 'images/2021/3-4'

            if not os.path.isdir(folderPath):
                print("Error - specified folderpath is not a directory. Use option d to see available folderpaths.")
            else:
                images = loadImages(folderPath)
                print('Length of images to be processed: %d' % len(images))
                
                # Update column names to Ring 0, Ring 1, etc.
                updateWorksheetNames()

                # Iterate over each image and save its percentages to excel sheet
                for i in range(len(images)):
                    print("Processing image {}...".format(i))
                    percentages = processImage(images[i], 0.012, 0.1, i, False)
                    savePercentages(percentages, i)
                
                # Save excell sheet to ../trainingData/ folder with its respective file name
                folderPath = folderPath.split("/")
                folderPath.pop(0)
                print("making excel sheet")
                newFileName = "LocalTest"
                #newFileName = "-".join(folderPath)
                workbook.save(filename=f"./trainingData/{newFileName}.xlsx")


        # Handles the t option used for testing
        elif currOption == "t":
            testImagePath = './images/2021/testImgs/img19.jpg'

            # Console message for User
            print("---------------\nEnter 'q' to quit into either threshold value.")
            threshLow = input('Enter the threshold Low Limit: ')
            threshHigh = input('Enter the threshold High Limit: ')
            interval = input('Enter the interval that either the high or low will increment by: ')
            numIterations = input('Enter the number of iterations: ')
            lowOrHigh = input("Type 'l' for incrementing on the low threshold. Type 'h' for incrementing on the high threshold: ")
            while threshLow != "q" and threshHigh != "q":
                try:
                    threshLow = float(threshLow)
                    threshHigh = float(threshHigh)
                    numIterations = int(numIterations)
                    print("Num Iter: {}".format(numIterations))
                    interval = float(interval)

                    for i in range(numIterations):
                        tempThreshLow = threshLow
                        tempThreshHigh = threshHigh
                        if lowOrHigh == "l":
                            tempThreshLow = threshLow + (interval * i)
                        elif lowOrHigh == "h":
                            tempThreshHigh = threshHigh + (interval * i)

                        #MR: Not sure if image count should be 0, but I think that is fine because we aren't really using testing function anymore
                        processImage(testImagePath, tempThreshLow, tempThreshHigh, 0, True)
                    
                    cv2.waitKey()
                    cv2.destroyAllWindows()

                except Exception as e:
                    print("\nERROR:\n")
                    print(e)
                    print("\nCould not properly parse input. Quitting...\n")
                    threshLow = "q"
                
                print("---------------\nEnter 'q' to quit into either threshold value.")
                threshLow = input('Enter the threshold Low Limit: ')
                threshHigh = input('Enter the threshold High Limit: ')
                interval = input('Enter the interval that either the high or low will increment by: ')
                numIterations = input('Enter the number of iterations: ')
                lowOrHigh = input("Type 'l' for incrementing on the low threshold. Type 'h' for incrementing on the high threshold: ")


        # Handles the d option used for displaying folders
        elif currOption == "d":
            pass


        # Handles unknown flags
        else:
            print("Unknown option. Try again.\n")
    
    
    exit(0)    



ModuleNotFoundError: ignored

In [ ]:
from pysolar.solar import *
import math
import os
import shutil
from PIL import Image
import cv2
import numpy as np
import datetime
import pytz


""" old forward and inverse mappings are commented
    out, see correctFisheye for accurate function
    (ideal scale factor is ~3.5 to 3.75 )"""

# def elipseToSquareCoordinates(u, v):
#   u2 = u * u
#   v2 = v * v
#   twosqrt2 = 2.0 * math.sqrt(2.0)
#   subtermx = 2.0 + u2 - v2
#   subtermy = 2.0 - u2 + v2
#   termx1 = subtermx + u * twosqrt2
#   termx2 = subtermx - u * twosqrt2
#   termy1 = subtermy + v * twosqrt2
#   termy2 = subtermy - v * twosqrt2
#   x = 0.5 * math.sqrt(termx1) - 0.5 * math.sqrt(termx2)
#   y = 0.5 * math.sqrt(termy1) - 0.5 * math.sqrt(termy2)
#   return (x, y)


# def squareToElipseCoordinates(x, y):
#   u = x * math.sqrt(1.0 - y * y / 2.0)
#   v = y * math.sqrt(1.0 - x * x / 2.0)
#   return (u, v)


def distance(x,y):
  return math.sqrt(x * x + y * y)


def correctFisheye(srcSize, destSize, dx, dy, factor):
  """ returns a tuple of source coordinates (sx,sy)
      (note: values can be out of range) """
  # convert dx,dy to relative coordinates
  rx, ry = dx - (destSize[0] / 2), dy - (destSize[1] / 2)
  # calculate theta
  r = distance(rx,ry) / (distance(srcSize[0], srcSize[1]) / factor)
  if 0 == r:
      theta = 1.0
  else:
      theta = math.atan(r) / r
  # back to absolute coordinates
  sx, sy = (srcSize[0] / 2) + theta * rx, (srcSize[1] / 2) + theta * ry
  return (int(round(sx)),int(round(sy)))


def withinBounds(x, y, img):
  height, width = img.shape[:2]
  return y >= 0 and y < height and x >= 0 and x < width


def flattenImage(imgName, factor):
  img = cv2.imread("./images/" + imgName, cv2.IMREAD_COLOR)
  height, width, channels = img.shape
  flattenedImg = np.zeros((height, height, channels), np.uint8)
  # xScale = 1 / width
  # yScale = 1 /height
  for y in range(height):
    # centerY = round(height / 2)
    # dy = y - centerY
    # dyScaled = 2 * dy / height
    for x in range(height):
      # centerX = round(width / 2)
      # dx = x - centerX
      # dxScaled = 2 * dx / width
      srcX, srcY = correctFisheye((width, height), (height, height), x, y, factor)
      if withinBounds(srcX, srcY, img):
        flattenedImg[y, x] = img[srcY, srcX]

  cv2.imwrite("./flatImages/" + imgName[:-4] + "Flattened" + imgName[len(imgName) - 4:], flattenedImg)


# def printSunStats():
#   startTime = 600  # 10:00am expressed in minutes
#   latitude = 35.32
#   longitude = -120.69
#   for imageNum in range(89):
#     curTime = startTime + 5 * imageNum
#     hour, minute = convertMinToTime(curTime)
#     # naive date with no sense of timezone
#     date = datetime.datetime(2020, 5, 19, hour, minute, 0, 0)  
#     timezone = pytz.timezone("America/Los_Angeles")
#     # aware date now includes timezone info (accounts for daylight savings as well)
#     awareDate = timezone.localize(date)  
#     azimuth = get_azimuth(latitude, longitude, awareDate)
#     altitude = get_altitude(latitude, longitude, awareDate)    

#     print("")
#     print("img" + str(imageNum))
#     print(str(hour) + ":" + ("0" if minute < 10 else "") + str(minute))  # time in format HH:MM
#     print("azimuth angle: " + str(azimuth))
#     print("altitude angle: " + str(altitude))


def boundCoordinates(x, y, img):
  height, width = img.shape[:2]
  if (x >= width):
    x = width - 1
  elif (x < 0):
    x = 0
  if (y >= height):
    y = height - 1
  elif (y < 0):
    y = 0
  return (x, y)


def convertMinToTime(min):
    hour = min / 60
    minute = min % 60
    return math.floor(hour), math.floor(minute)


def getSolarAngles(imageNumber):
  startTime = 600  # 10:00am expressed in minutes
  latitude = 35.32
  longitude = -120.69
  curTime = startTime + 5 * imageNumber
  hour, minute = convertMinToTime(curTime)

  print(("0" + str(hour) if hour < 10 else str(hour)) + ":" + ("0" + str(minute) if minute < 10 else str(minute)))

  # naive date with no sense of timezone
  date = datetime.datetime(2020, 5, 19, hour, minute, 0, 0)  
  timezone = pytz.timezone("America/Los_Angeles")
  # aware date now includes timezone info (accounts for daylight savings as well)
  awareDate = timezone.localize(date)  
  azimuth = get_azimuth(latitude, longitude, awareDate)
  altitude = get_altitude(latitude, longitude, awareDate)
  return (azimuth, altitude)


def findSun(image, imageNumber):
  print("")

  azimuth, altitude = getSolarAngles(imageNumber)  # angles are in degrees
  
  print("img" + str(imageNumber))
  print("azimuth: " + str(azimuth))
  print("altitude: " + str(altitude))

    # convert to radians
  azimuth = (azimuth - 90) / 180 * math.pi
  altitude = altitude / 180 * math.pi

  height, width = image.shape[:2]
  xCenter = round(width / 2)
  yCenter = round(height / 2)
  xScale = math.cos(azimuth)
  yScale = math.sin(azimuth)
  zScale = math.cos(altitude)
  # distance from image edge to fisheye edge is ~255 pixels
  x = xCenter + round(xScale * zScale * (width - 255) / 2)
  y = yCenter + round(yScale * zScale * (width - 255) / 2)
  return boundCoordinates(x, y, image)


def main():
  for imageNumber in range(0, 89):
    image = cv2.imread("./images/img" + str(imageNumber) + ".jpg", cv2.IMREAD_COLOR)
    sunCoordinates = findSun(image, imageNumber)
    imageCopy = image.copy()
    imageCopy = cv2.circle(imageCopy, sunCoordinates, 20, (0, 0, 255), -1)
    center = (round(image.shape[1] / 2), round(image.shape[0] / 2))
    imageCopy = cv2.circle(imageCopy, center, 20, (0, 0, 255), -1)
    cv2.imwrite("./sunImages/img" + str(imageNumber) + "Sun" + ".jpg", imageCopy)

    # flattenImage("img" + str(imageNumber) + ".jpg", 3.75)


    # image = cv2.imread("./images/img" + str(imageNumber) + ".jpg", cv2.IMREAD_COLOR)
    # findSun(image, imageNumber)


if __name__ == "__main__":
  main()


In [ ]:
!pip install pysolar

     |████████████████████████████████| 51kB 4.1MB/s 


img0
azimuth: 99.16539643801367
altitude: 47.87474856103299

img1
azimuth: 100.07890584638295
altitude: 48.88015931005149

img2
azimuth: 101.01822832049325
altitude: 49.88270318998996

img3
azimuth: 101.98550284936401
altitude: 50.882022386934366

img4
azimuth: 102.98305861459149
altitude: 51.87772779889578

img5
azimuth: 104.01343583278573
altitude: 52.86939532049324

img6
azimuth: 105.07940889210506
altitude: 53.85656157130187

img7
azimuth: 106.18401137085105
altitude: 54.8387184570404

img8
azimuth: 107.33056341021431
altitude: 55.81530687112883

img9
azimuth: 108.52270329704544
altitude: 56.78571087385489

img10
azimuth: 109.76441947746709
altitude: 57.74924893727294

img11
azimuth: 111.06008755374826
altitude: 58.70516605510173

img12
azimuth: 112.41450808896906
altitude: 59.652623286549826

img13
azimuth: 113.8329463192081
altitude: 60.59068603343677

img14
azimuth: 115.32117346568882
altitude: 61.51831124023941

img15
azimuth: 116.88550757547799
altitude: 62.43433282601981

img16
azimuth: 118.53285074740259
altitude: 63.33744470105376

img17
azimuth: 120.27071982387554
altitude: 64.22618165136377

img18
azimuth: 122.10726845332763
altitude: 65.09889934364698

img19
azimuth: 124.05128813968508
altitude: 65.95375051751395

img20
azimuth: 126.11218627785604
altitude: 66.78866119983529

img21
azimuth: 128.2999232424811
altitude: 67.60130450627574

img22
azimuth: 130.62489554493808
altitude: 68.38907433160392

img23
azimuth: 133.09774702781715
altitude: 69.14906074412701

img24
azimuth: 135.7290847675679
altitude: 69.87802889906436

img25
azimuth: 138.52907371734437
altitude: 70.57240435258126

img26
azimuth: 141.50688723057004
altitude: 71.22826956611884

img27
azimuth: 144.67000103859593
altitude: 71.84137813718866

img28
azimuth: 148.02331871084655
altitude: 72.4071911285097

img29
azimuth: 151.56816471378158
altitude: 72.92094518453759

img30
azimuth: 155.30119791089382
altitude: 73.37775662858843

img31
azimuth: 159.21336201458206
altitude: 73.77276646307295

img32
azimuth: 163.28903770946755
altitude: 74.10132545900953

img33
azimuth: 167.5055927271071
altitude: 74.35921096202014

img34
azimuth: 171.83352270553885
altitude: 74.54285876518968

img35
azimuth: 176.23732008537186
altitude: 74.64958583167046

img36
azimuth: 180.6770940171735
altitude: 74.67777480333888

img37
azimuth: 185.11078558916617
altitude: 74.62699208444918

img38
azimuth: 189.4967080416186
altitude: 74.49802051609015

img39
azimuth: 193.79602609364895
altitude: 74.29280080019021

img40
azimuth: 197.9748248083195
altitude: 74.01429184660218

img41
azimuth: 202.00551958313218
altitude: 73.66627230326067

img42
azimuth: 205.86751036458193
altitude: 73.2531121236188

img43
azimuth: 209.54713403872165
altitude: 72.77954279235541

img44
azimuth: 213.03707650865636
altitude: 72.2504486576118

img45
azimuth: 216.33545065552502
altitude: 71.6706925311145

img46
azimuth: 219.4447189338943
altitude: 71.04498303743928

img47
azimuth: 222.37062370893452
altitude: 70.37778086316513

img48
azimuth: 225.1212112766657
altitude: 69.67324093488469

img49
azimuth: 227.70600557102262
altitude: 68.93518306617933

img50
azimuth: 230.13534750761505
altitude: 68.1670836906168

img51
azimuth: 232.41989073409638
altitude: 67.37208263348235

img52
azimuth: 234.57023311525217
altitude: 66.5530001303042

img53
azimuth: 236.5966610659772
altitude: 65.71235977597307

img54
azimuth: 238.50898542165456
altitude: 64.8524131526571

img55
azimuth: 240.31644035555476
altitude: 63.975166888896275

img56
azimuth: 242.02763549750912
altitude: 63.08240741786934

img57
azimuth: 243.6505401710912
altitude: 62.17572546904116

img58
azimuth: 245.1924917867787
altitude: 61.25653860957524

img59
azimuth: 246.66022053897092
altitude: 60.32611125470481

img60
azimuth: 248.05988335916476
altitude: 59.38557257852269

img61
azimuth: 249.39710215854262
altitude: 58.43593291909929

img62
azimuth: 250.6770041021059
altitude: 57.47809846455709

img63
azimuth: 251.90426370702755
altitude: 56.51288294300091

img64
azimuth: 253.08314110371032
altitude: 55.54101970208952

img65
azimuth: 254.21752082580966
altitude: 54.563170483860176

img66
azimuth: 255.31094643262097
altitude: 53.57993428915633

img67
azimuth: 256.36665270650997
altitude: 52.59185505725942

img68
azimuth: 257.3875957162451
altitude: 51.599427898708

img69
azimuth: 258.37648016649155
altitude: 50.60310450525018

img70
azimuth: 259.3357836561028
altitude: 49.6032983953216

img71
azimuth: 260.2677784951265
altitude: 48.60038969041505

img72
azimuth: 261.1745525923229
altitude: 47.594728013520765

img73
azimuth: 262.0580264444246
altitude: 46.586636947544584

img74
azimuth: 262.9199701701177
altitude: 45.57641606860334

img75
azimuth: 263.7620176575814
altitude: 44.56434398740513

img76
azimuth: 264.5856793964001
altitude: 43.55068095156236

img77
azimuth: 265.3923544330168
altitude: 42.53567064764468

img78
azimuth: 266.183341108552
altitude: 41.519541768583444

img79
azimuth: 266.95984629636985
altitude: 40.5025099295322

img80
azimuth: 267.7229936117543
altitude: 39.484779551580445

img81
azimuth: 268.4738318019051
altitude: 38.46654422628563

img82
azimuth: 269.2133408322649
altitude: 37.44798900119084

img83
azimuth: 269.9424388277336
altitude: 36.429290497679496

img84
azimuth: 270.6619873891842
altitude: 35.410618304546695

img85
azimuth: 271.3727964933818
altitude: 34.39213614385697

img86
azimuth: 272.075629296355
altitude: 33.37400242222089

img87
azimuth: 272.7712064935648
altitude: 32.356370716479034

img88
azimuth: 273.4602099158524
altitude: 31.33939075665524

```
# This is formatted as code
```

